# Exam Project Notebook

### Authors: 
### Bjørn Bremholm
### Laura Zeeper
### Christoffer Gade

In [317]:
# Import Packages 

import numpy as np 
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import tweepy
import webbrowser
import time

# Suppress warning
from urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

# Text analysis
import nltk # NLTK: A basic, popular NLP package. 


# Header to be used in request (contact info)
header = {'Contact' : 'Christofferfoldager@gmail.com'}

## 1. Section - Gather list of twitter accounts 

Description: In this section we will collect a dataframe containing infomation of danish MPs including their twitter accounts...

Websites: 
1. https://www.ft.dk/da/medlemmer/mandatfordelingen - used to collect parties and the official MP's of each party
2. https://filip.sdu.dk/twitter/politikere/ - used to collect the twitter accounts for each MP



In [24]:
# Collect the address link of each party
url = 'https://www.ft.dk/da/medlemmer/mandatfordelingen'

header['Info'] = 'Collecting a list of MP for research project'
page = requests.get(url, verify=False, headers=header)
soup = BeautifulSoup(page.text, 'html.parser')

# Get the links
data = soup.find_all('table')[0]
aref = data.find_all('a')
member_nr = data.find_all('div', {'class':'member-container'})

# Create a list with all relevant infomation 
Parties = []

# defining red and blue 
Red = ['Socialdemokratiet (S)','Socialistisk Folkeparti (SF)','Radikale Venstre (RV)','Enhedslisten (EL)','Alternativet (ALT)']
Blue = ['Det Konservative Folkeparti (KF)', 'Venstre (V)', 'Dansk Folkeparti (DF)', 'Det Konservative Folkeparti (KF)', 'Liberal Alliance (LA)', 'Nye Borgerlige (NB)', 'Kristendemokraterne (KD)']

for i in range(len(aref)):
    if aref[i].text in Red:
        Color = 'Red'
    elif aref[i].text in Blue:
        Color = 'Blue'
    else:
        Color = None
    Parties.append([aref[i].text,Color, aref[i].get('href'),member_nr[i].text])

# Creates a Dataframe    
df_Parties = pd.DataFrame(Parties, columns=['Party','Color','AddressLink','Members'])
df_Parties

,Party,Color,AddressLink,Members
0,Socialdemokratiet (S),Red,/searchResults.aspx?sortedDescending=false&par...,49
1,Venstre (V),Blue,/searchResults.aspx?sortedDescending=false&par...,39
2,Dansk Folkeparti (DF),Blue,/searchResults.aspx?sortedDescending=false&par...,16
3,Socialistisk Folkeparti (SF),Red,/searchResults.aspx?sortedDescending=false&par...,15
4,Radikale Venstre (RV),Red,/searchResults.aspx?sortedDescending=false&par...,14
5,Enhedslisten (EL),Red,/searchResults.aspx?sortedDescending=false&par...,13
6,Det Konservative Folkeparti (KF),Blue,/searchResults.aspx?sortedDescending=false&par...,13
7,Nye Borgerlige (NB),Blue,/searchResults.aspx?sortedDescending=false&par...,4
8,Liberal Alliance (LA),Blue,/searchResults.aspx?sortedDescending=false&par...,3
9,Alternativet (ALT),Red,/searchResults.aspx?sortedDescending=false&par...,1


In [118]:
# Collect the names of each MP 
ID_list = []

for i in range(len(df_Parties)): # Loop for each party
    Party = df_Parties['Party'][i]
    Color = df_Parties['Color'][i]

    #Get the relevant party page
    link = df_Parties['AddressLink'][i]
    url = f'http://ft.dk{link}&page=1&sortedBy=&pageSize=50'
    # time.sleep(1)
    page = requests.get(url, verify=False, headers=header)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Get the names 
    tables = soup.find_all('tr',{'tabindex':0})

    for i in range(len(tables)): # Loop for each member of the given party 
        FirstName = tables[i].find_all('td')[1].text
        LastName = tables[i].find_all('a')[0].text
        Name = FirstName + ' ' + LastName
        MP = tables[i].find_all('td')[4].text

        # Create a first, last and middle name (for use in merge)
        Name_list = re.findall('\w+',Name)
        First_Name = Name_list[0]
        Last_Name = Name_list[-1]
        Middel_Name = ' '.join(Name_list[1:-1])

        ID_list.append([Name,Party,Color,MP,First_Name,Last_Name,Middel_Name])

df_Name = pd.DataFrame(ID_list, columns=['Name','Party','Color','MP','First Name','Last Name','Middel Name'])

# Give color to the people out of party
Red_Name = ['Susanne Zimmer','Sikandar Siddique','Uffe Elbæk'] 
Blue_Name = ['Simon Emil Ammitzbøll-Bille', 'Lars Løkke Rasmussen', 'Orla Østerby', 'Inger Støjberg']

df_Name['Color'] = np.where((df_Name['Name'].isin(Red_Name)),'Red',df_Name['Color'])
df_Name['Color'] = np.where((df_Name['Name'].isin(Blue_Name)),'Blue',df_Name['Color'])

df_Name

,Name,Party,Color,MP,First Name,Last Name,Middel Name
0,Ida Auken,Socialdemokratiet (S),Red,Medlem af Folketinget,Ida,Auken,
1,Trine Bramsen,Socialdemokratiet (S),Red,Medlem af Folketinget,Trine,Bramsen,
2,Bjørn Brandenborg,Socialdemokratiet (S),Red,Medlem af Folketinget,Bjørn,Brandenborg,
3,Jeppe Bruus,Socialdemokratiet (S),Red,Medlem af Folketinget,Jeppe,Bruus,
4,Morten Bødskov,Socialdemokratiet (S),Red,Medlem af Folketinget,Morten,Bødskov,
5,Lennart Damsbo-Andersen,Socialdemokratiet (S),Red,Medlem af Folketinget,Lennart,Andersen,Damsbo
6,Kaare Dybvad Bek,Socialdemokratiet (S),Red,Medlem af Folketinget,Kaare,Bek,Dybvad
7,Benny Engelbrecht,Socialdemokratiet (S),Red,Medlem af Folketinget,Benny,Engelbrecht,
8,Camilla Fabricius,Socialdemokratiet (S),Red,Medlem af Folketinget,Camilla,Fabricius,
9,Mette Frederiksen,Socialdemokratiet (S),Red,Medlem af Folketinget,Mette,Frederiksen,


In [119]:
# Collect the twitter accounts 
url = 'https://filip.sdu.dk/twitter/politikere/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

data = soup.find_all('div', {'class':'col-md-6 show_tweet show_user'}) # Finds the list of all politicians twitter account

Twitter_list = []

for i in range(len(data)):
    info_i = data[i].find_all('div', {'class':'media-body'})
    twitter_id = info_i[0].find('h3').text.split()
    person_id = info_i[0].find('small').text.split(' \nmed ')

    Name = person_id[0]
    Tag = twitter_id[1]
    Følgere = re.search('[0-9.]+[0-9]|\d', person_id[1]).group()

    # Create a first, last and middle name (for merge purpose)
    Name_list = re.findall('\w+',Name)
    First_Name = Name_list[0]
    Last_Name = Name_list[-1]
    Middel_Name = ' '.join(Name_list[1:-1])

    Twitter_list.append([Name,Tag,Følgere, First_Name, Last_Name,Middel_Name])

df_Twitter_id = pd.DataFrame(Twitter_list, columns=['Name','Twitter_id','Følgere','First Name','Last Name','Middel Name'])
df_Twitter_id.head(5)

,Name,Twitter_id,Følgere,First Name,Last Name,Middel Name
0,Margrethe Vestager,@vestager,295.327,Margrethe,Vestager,
1,Lars Løkke Rasmussen,@larsloekke,198.345,Lars,Rasmussen,Løkke
2,Pernille Skipper,@PSkipperEL,82.871,Pernille,Skipper,
3,Ida Auken,@IdaAuken,73.926,Ida,Auken,
4,Kristian Jensen,@Kristian_Jensen,62.946,Kristian,Jensen,


In [120]:
# First merge effort
def merge_df(df_name,df_twitter):
    # Merge on first and last name 
    df_new = pd.merge(df_name,df_twitter[['First Name', 'Last Name','Twitter_id','Følgere']],  how='left', on=['First Name','Last Name'])
    # Merge on first and middel name
    df_new1 = pd.merge(df_name, df_twitter[['First Name', 'Last Name','Twitter_id','Følgere']],  how='left', left_on = ['First Name','Middel Name'], right_on=['First Name', 'Last Name'])

    df_new['Twitter_id'] = df_new['Twitter_id'].fillna(df_new1['Twitter_id'])
    df_new['Følgere'] = df_new['Følgere'].fillna(df_new1['Følgere'])

    # We will still miss a few Twitter_id (human error or no twitter account)
    df_new.replace(float('NaN'),'None',inplace=True)

    return df_new

df_info = merge_df(df_Name,df_Twitter_id)
twitter_missing = len(df_info.loc[df_info['Twitter_id'] == 'None'])
print ('We are missing: '+ str(twitter_missing) + ' twitter accounts')


We are missing: 32 twitter accounts


In [121]:
manuel_handles = {'Kaare Dybvad Bek':'@KaareDybvad', 'Karin Gaardsted' : '@KarinGaardsted', 'Ane Halsboe-Jørgensen':'@AneHalsboe', 
                    'Christian Rabjerg Madsen' : '@RabjergMadsen', 'Lars Aslan Rasmussen':'@lars_aslan', 'Pernille Rosenkrantz-Theil' : '@RosenkrantzT',
                    'Kasper Roug':'@KasperRoug', 'Mads  Fuglede':'@madsfuglede', 'Peter Juel-Jensen':'@PeterJuelJensen',
                    'Stén Knuth':'@Sten_Knuth',  'Lars Christian Lilleholt':'@larsclilleholt', 'Kristian Pihl Lorentzen':'@kplorentzen',
                    'Torsten Schack Pedersen':'@Torstenschack', 'Lise Bech':'@LiseBech', 'Jens Henrik Thulesen Dahl':'@JThulesen', 
                    'Mette Hjermind Dencker':'@dfmehd_mette', 'Kirsten Normann Andersen':'@KirstenNormann', 'Karina Lorentzen Dehnhardt':'@MF_K_Lorentzen',
                    'Charlotte Broman Mølbæk':'@charlottebroman', 'Rasmus Nordqvist':'@rasmusnordqvist', 'Trine Torp':'@TrineTorp', 
                    'Sofie Carsten Nielsen':'@sofiecn', 'Rasmus Helveg Petersen':'@rasmushelveg', 'Victoria Velasquez':'@VictoriaV_EL', 
                    'Katarina Ammitzbøll':'@Ammitzboell_K', 'Brigitte Klintskov Jerkel':'@JerkelK', 'Aki-Matilda Høegh-Dam': '@AkiMati_Siumut',
                    'Sjúrður Skaale': '@SjurSkaale', 'Uffe Elbæk':'@uffeelbaek'}

In [122]:
for key in manuel_handles:
    index = df_info[df_info['Name'] == key].index
    df_info.iloc[index[0],7] = manuel_handles[key]

twitter_missing = len(df_info.loc[df_info['Twitter_id'] == 'None'])
print ('We are missing: '+ str(twitter_missing) + ' twitter accounts')
df_info.head()

We are missing: 16 twitter accounts


,Name,Party,Color,MP,First Name,Last Name,Middel Name,Twitter_id,Følgere
0,Ida Auken,Socialdemokratiet (S),Red,Medlem af Folketinget,Ida,Auken,,@IdaAuken,73.926
1,Trine Bramsen,Socialdemokratiet (S),Red,Medlem af Folketinget,Trine,Bramsen,,@Trinebramsen,13.483
2,Bjørn Brandenborg,Socialdemokratiet (S),Red,Medlem af Folketinget,Bjørn,Brandenborg,,@BjBrandenborg,2.277
3,Jeppe Bruus,Socialdemokratiet (S),Red,Medlem af Folketinget,Jeppe,Bruus,,@JeppeBruus,4.066
4,Morten Bødskov,Socialdemokratiet (S),Red,Medlem af Folketinget,Morten,Bødskov,,@mfMorten,15.324


In [130]:
df_data = df_info.copy()

# Dropping columns
Columns = ['First Name', 'Last Name','Middel Name']
df_data = df_data.drop(columns=Columns, axis=1)

# Dropping those without color
df_data = df_data[df_data.Twitter_id != 'None']


# Dropping MPs from greenland and faroe islands
df_data = df_data[df_data.Color != 'None']

# Reindex
df_data = df_data.reset_index(drop=True)

# Create y binary
df_data['y'] = np.where((df_data['Color'] == 'Blue'),1,0)
# df_Name['Color'] = np.where((df_Name['Name'].isin(Blue_Name)),'Blue',df_Name['Color'])

pd.set_option('display.max_rows', None)
df_data


,Name,Party,Color,MP,Twitter_id,Følgere,y
0,Ida Auken,Socialdemokratiet (S),Red,Medlem af Folketinget,@IdaAuken,73.926,0
1,Trine Bramsen,Socialdemokratiet (S),Red,Medlem af Folketinget,@Trinebramsen,13.483,0
2,Bjørn Brandenborg,Socialdemokratiet (S),Red,Medlem af Folketinget,@BjBrandenborg,2.277,0
3,Jeppe Bruus,Socialdemokratiet (S),Red,Medlem af Folketinget,@JeppeBruus,4.066,0
4,Morten Bødskov,Socialdemokratiet (S),Red,Medlem af Folketinget,@mfMorten,15.324,0
5,Kaare Dybvad Bek,Socialdemokratiet (S),Red,Medlem af Folketinget,@KaareDybvad,7.432,0
6,Benny Engelbrecht,Socialdemokratiet (S),Red,Medlem af Folketinget,@BennyEngelbrech,20.829,0
7,Camilla Fabricius,Socialdemokratiet (S),Red,Medlem af Folketinget,@CamFabricius,2.147,0
8,Mette Gjerskov,Socialdemokratiet (S),Red,Medlem af Folketinget,@MetteGjerskov,12.388,0
9,Karin Gaardsted,Socialdemokratiet (S),Red,Stedfortræder,@KarinGaardsted,None,0


## Section 2 - Collecting Tweets


In [133]:
# Get authentication by twitter and access to their API 

callback_url = 'oob'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret, callback_url)

try:
    redirect_url = auth.get_authorization_url()
except tweepy.TweepError:
    print('Error! Failed to get request token.')

webbrowser.open(redirect_url)

user_pin_input = input('What is the pin value? ')
auth.get_access_token(user_pin_input)
api = tweepy.API(auth)

In [324]:
# Function gathering the tweets from a specific user/handle
def extract_tweets(handle,name):

    # Input: Handle/user (e.g. @JeppeKofod) which is used to collect tweets
    #
    # Output: Dataframe containing tweets with attaching information 

    # tweets = api.user_timeline(screen_name=handle,count=40,exclude_replies=True,include_rts=False,tweet_mode="extended") 
    tweets = tweepy.Cursor(api.user_timeline,screen_name = handle, tweet_mode='extended', include_rts=False, exclude_replies=True).items()
    tweets_data = []

    for tweet in tweets:
        date = tweet.created_at
        author = tweet.author.screen_name
        likes = tweet.favorite_count
        retweets = tweet.retweet_count
        source = tweet.source
        text = tweet.full_text
        lang = tweet.lang

        tweets_data.append([date,author,name,text,lang,likes,retweets,source])

    df_tweets = pd.DataFrame(tweets_data, columns=['Date','Author','Name','Tweet','Language','Likes','Retweets','Source'])
    return df_tweets 

# Get dataframe of tweets and updated df_data
def get_tweets_df(df):
    
    df = df.reset_index(drop=True)

    df_out = pd.DataFrame()
    tweet_count = []

    for i in range(len(df)):
        
        # Find user
        user = df['Twitter_id'][i]
        name = df['Name'][i]
        
        # Collects tweets from user
        df_tweets = extract_tweets(user,name)
        df_out = df_out.append(df_tweets)

        # Collect total of tweets
        tweet_count.append(len(df_tweets))
        
        # Request limit time sleeper
        time.sleep(90)
    
    df_out = df_out.reset_index(drop=True)
    df['Tweet Count'] = tweet_count

    return df_out,df


In [329]:
# NEED LANGUAGE
df1 = df_data[80:110].copy()

# Get twitter data
df_twitter_data, df_Name_list = get_tweets_df(df1)

# Process tweets
df_tweet_processed = process_tweets_text(df_twitter_data)


df_tweet_processed.to_csv('Twitter_Processed3.csv')
df_Name_list.to_csv('Name_id_list3.csv')
df_twitter_data.to_csv('twitter_data3.csv')

df_twitter_data.head(20)

,Date,Author,Name,Tweet,Language,Likes,Retweets,Source
0,2021-05-12 08:10:27,fatmaoektem,Fatma Øktem,Vi skal have et opgør med det ekstremt kønsopd...,da,40,6,Twitter Web App
1,2021-04-30 20:36:54,fatmaoektem,Fatma Øktem,Jeg undrer mig også over ministeren ikke delto...,da,8,2,Twitter for iPhone
2,2021-04-30 20:30:43,fatmaoektem,Fatma Øktem,https://t.co/fyyOPkNc0G,und,2,1,Twitter for iPhone
3,2021-03-27 16:10:20,fatmaoektem,Fatma Øktem,Gerne mere ledelse og gerne mindre kontrol og ...,da,13,2,Twitter for iPhone
4,2021-03-08 09:21:24,fatmaoektem,Fatma Øktem,"Jeg takker de stærke kvinder og mænd, der kæmp...",da,118,6,Twitter for iPhone
5,2021-02-19 20:11:07,fatmaoektem,Fatma Øktem,Carsten Hansen blev “straffet” med en minister...,da,54,10,Twitter for iPhone
6,2021-02-09 09:23:59,fatmaoektem,Fatma Øktem,Alt for længe har vi tildelt statsborgerskab t...,da,96,9,Twitter for iPhone
7,2020-11-09 19:37:36,fatmaoektem,Fatma Øktem,"Enig, det er vigtigt og det er ret alvorligt. ...",da,16,4,Twitter for iPhone
8,2020-10-08 08:04:07,fatmaoektem,Fatma Øktem,Vi vil ikke tilbage - vi vil ind i fremtiden @...,da,14,4,Twitter for iPhone
9,2020-08-29 11:00:23,fatmaoektem,Fatma Øktem,Stærkt af Sofie Linde at stå frem. Det ikke i ...,da,7,1,Twitter for iPhone


In [330]:
df_Name_list

,Name,Party,Color,MP,Twitter_id,Følgere,y,Tweet Count
0,Fatma Øktem,Venstre (V),Blue,Medlem af Folketinget,@fatmaoektem,4.292,1,576
1,Karina Adsbøl,Dansk Folkeparti (DF),Blue,Medlem af Folketinget,@AdsbolAdsbl,2.885,1,1450
2,Lise Bech,Dansk Folkeparti (DF),Blue,Medlem af Folketinget,@LiseBech,None,1,2228
3,Liselott Blixt,Dansk Folkeparti (DF),Blue,Medlem af Folketinget,@Blixt22,6.843,1,424
4,Bent Bøgsted,Dansk Folkeparti (DF),Blue,Medlem af Folketinget,@BentGsted,136,1,6
5,Jens Henrik Thulesen Dahl,Dansk Folkeparti (DF),Blue,Medlem af Folketinget,@JThulesen,None,1,226
6,Kristian Thulesen Dahl,Dansk Folkeparti (DF),Blue,Medlem af Folketinget,@Kristianthdahl,16.827,1,636
7,Mette Hjermind Dencker,Dansk Folkeparti (DF),Blue,Medlem af Folketinget,@dfmehd_mette,1.120,1,181
8,Søren Espersen,Dansk Folkeparti (DF),Blue,Medlem af Folketinget,@espersendf,12.040,1,1199
9,Dennis Flydtkjær,Dansk Folkeparti (DF),Blue,Medlem af Folketinget,@flydtkjaer,3.282,1,296


In [320]:
df_Name_list

,Name,Party,Color,MP,Twitter_id,Følgere,y,Tweet Count
0,Ida Auken,Socialdemokratiet (S),Red,Medlem af Folketinget,@IdaAuken,73.926,0,879
1,Trine Bramsen,Socialdemokratiet (S),Red,Medlem af Folketinget,@Trinebramsen,13.483,0,946
2,Bjørn Brandenborg,Socialdemokratiet (S),Red,Medlem af Folketinget,@BjBrandenborg,2.277,0,312
3,Jeppe Bruus,Socialdemokratiet (S),Red,Medlem af Folketinget,@JeppeBruus,4.066,0,592
4,Morten Bødskov,Socialdemokratiet (S),Red,Medlem af Folketinget,@mfMorten,15.324,0,2123
5,Kaare Dybvad Bek,Socialdemokratiet (S),Red,Medlem af Folketinget,@KaareDybvad,7.432,0,38
6,Benny Engelbrecht,Socialdemokratiet (S),Red,Medlem af Folketinget,@BennyEngelbrech,20.829,0,1160
7,Camilla Fabricius,Socialdemokratiet (S),Red,Medlem af Folketinget,@CamFabricius,2.147,0,663
8,Mette Gjerskov,Socialdemokratiet (S),Red,Medlem af Folketinget,@MetteGjerskov,12.388,0,1521
9,Karin Gaardsted,Socialdemokratiet (S),Red,Stedfortræder,@KarinGaardsted,None,0,107


In [304]:
# df_tweet_processed

In [311]:
nr = 3
text = df_twitter_data['Tweet'][nr]
text

def preprocess(text):
    # lemmas = text.lower()
    text = re.sub('https:[\s\S]*$','', text) # remove links
    text = re.sub(r'[^\w\#\s]','', text) # should we leave hashtag or not

    tokens = nltk.TweetTokenizer().tokenize(text.lower())

    # tokens = nltk.word_tokenize(text.lower())
    # wnl = nltk.WordNetLemmatizer()
    # lemmas = [wnl.lemmatize(t) for t in tokens]

    stop_words_list = nltk.corpus.stopwords.words("danish")
    lemmas = [i for i in tokens if i not in stop_words_list]

    return lemmas # return a list of stems/lemmas


def process_tweets_text(df):

    tweet_process = []


    for i in range(len(df['Tweet'])):
        text = df['Tweet'][i]

        tweet_process.append([df['Date'][i],df['Name'][i],preprocess(text),df['Language'][i]])

    df_out = pd.DataFrame(tweet_process, columns=['Date','Name','Tweet Processed','Language'])
    
    return df_out



print(text)
print(preprocess(text))


Svært ikke at være åbenhjertig ovor for så skøn en samtalepartner som Flemming Møldrup. Lyt med her #dkpol #dkgreen #stress @Heartbeatsdk @fmpldrup https://t.co/kjvSlhyyVY
['svært', 'åbenhjertig', 'ovor', 'så', 'skøn', 'samtalepartner', 'flemming', 'møldrup', 'lyt', '#dkpol', '#dkgreen', '#stress', 'heartbeatsdk', 'fmpldrup']


In [312]:
df_tweet_processed = process_tweets_text(df_twitter_data)
# df_twitter_data
df_tweet_processed

,Date,Name,Tweet Processed,Language
0,2021-06-23 14:20:48,Ida Auken,"[grinet, flere, dage, så, får, passer, desværr...",da
1,2021-06-12 17:31:49,Ida Auken,"[uefa, meddeler, eriksen, stabiliseret, hurra,...",no
2,2021-06-12 17:26:59,Ida Auken,"[sandt, #em2020]",da
3,2021-06-09 08:36:50,Ida Auken,"[svært, åbenhjertig, ovor, så, skøn, samtalepa...",da
4,2021-06-06 12:18:23,Ida Auken,[tillykke],da
5,2021-06-03 08:24:02,Ida Auken,"[skrevet, lidt, område, virkelig, halter, bagu...",da
6,2021-06-02 07:38:40,Ida Auken,"[lidt, år, taget, beslutninger, bringer, halvd...",da
7,2021-05-26 09:54:50,Ida Auken,"[endnu, set, dokumentaren, 7030, klimalovens, ...",da
8,2021-05-26 07:41:13,Ida Auken,"[getting, ready, talk, about, #climate, and, s...",en
9,2021-05-25 09:45:06,Ida Auken,"[what, a, brilliant, idea, tomhfh, oh, wait, w...",en
